# AKS 클러스터 실습 가이드

> **실습 환경**: 이 노트북은 Dev Container(GitHub Codespaces 또는 VS Code)에서 실행하도록 구성되어 있습니다. Azure CLI, kubectl 등 필요한 도구가 자동으로 설치되어 있습니다.

> **사전 준비**: Azure 구독과 이전 실습에서 생성한 ACR이 필요합니다.

이 노트북에서는 Azure Kubernetes Service(AKS) 클러스터 생성, 쿠버네티스 및 AKS의 기본 개념, 컨테이너 이미지 배포, 실행, 확장, 모니터링 실습을 진행합니다.

## 1. 쿠버네티스와 AKS 기본 개념 및 이점

### 쿠버네티스란?
쿠버네티스(Kubernetes)는 컨테이너화된 애플리케이션의 배포, 확장, 관리를 자동화하는 오픈소스 플랫폼입니다.

#### 쿠버네티스의 주요 이점
- **자동화된 배포 및 확장**
- **자가 치유(Self-healing)**
- **서비스 디스커버리 및 로드밸런싱**
- **롤링 업데이트 및 롤백**

### AKS란?
AKS(Azure Kubernetes Service)는 Azure에서 제공하는 관리형 쿠버네티스 서비스로, 클러스터 관리의 복잡성을 줄여줍니다.

#### AKS의 주요 이점
- **간편한 클러스터 관리**
- **Azure와의 통합**
- **자동 업그레이드 및 패치**

## 2. AKS 클러스터 생성 실습

Azure CLI를 사용하여 AKS 클러스터를 생성합니다.

> **GitHub Codespaces**: Azure CLI가 이미 설치되어 있으므로 바로 사용 가능합니다.

### 2.1 Azure 로그인

먼저 Azure에 로그인합니다.

In [ ]:
# 공통 설정 파일 import
from config import *

# 현재 설정 확인
print_config()

In [ ]:
%%bash
# Azure 로그인 세션 확인
if az account show &>/dev/null; then
  echo "✅ 이미 Azure에 로그인되어 있습니다."
  echo "   현재 구독: $(az account show --query name -o tsv)"
else
  echo "🔐 Azure 로그인이 필요합니다. 로그인을 진행합니다..."
  az login --output none
  
  # 첫 번째 구독을 자동으로 선택
  SUBSCRIPTION_ID=$(az account list --query "[0].id" -o tsv)
  az account set --subscription $SUBSCRIPTION_ID
  
  echo "✅ Azure 로그인 완료"
  echo "   사용 중인 구독: $(az account show --query name -o tsv)"
fi

### 2.2 리소스 그룹 확인

config.py에서 불러온 변수를 사용하여 리소스 그룹 존재 여부를 확인합니다.

In [ ]:
import subprocess

print(f"📝 config.py에서 불러온 변수:")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   AKS 클러스터: {AKS_CLUSTER_NAME}")
print()

# 리소스 그룹 존재 확인
result = subprocess.run(
    ["az", "group", "show", "--name", RESOURCE_GROUP],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(f"✅ 리소스 그룹이 존재합니다: {RESOURCE_GROUP}")
    subprocess.run(
        ["az", "group", "show", "--name", RESOURCE_GROUP, 
         "--query", "{Name:name, Location:location}", "--output", "table"]
    )
else:
    print(f"❌ 리소스 그룹이 존재하지 않습니다: {RESOURCE_GROUP}")
    print("   1번 노트북을 먼저 실행해주세요.")

### 2.3 AKS 클러스터 생성 (관리형 Prometheus 및 Grafana 활성화)

AKS 클러스터 생성 시 Azure Monitor의 관리형 Prometheus와 Grafana를 함께 활성화합니다.

> **참고**: 클러스터 생성에는 약 5-10분 정도 소요됩니다.

In [ ]:
import subprocess

# 위 셀에서 설정한 변수 사용
print("🚀 AKS 클러스터 생성을 시작합니다...")
print("⏱️  예상 소요 시간: 5-10분")
print()
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   클러스터 이름: {AKS_CLUSTER_NAME}")
print()

# 관리형 Prometheus 및 Grafana가 활성화된 AKS 클러스터 생성
result = subprocess.run([
    "az", "aks", "create",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-count", "2",
    "--enable-managed-identity",
    "--generate-ssh-keys",
    "--verbose"
])

if result.returncode == 0:
    print()
    print("✅ 클러스터 생성 완료!")
    print()
    
    # 클러스터 생성 상태 확인
    print("📋 클러스터 목록:")
    subprocess.run(["az", "aks", "list", "--resource-group", RESOURCE_GROUP, "--output", "table"])
    
    print()
    print("📊 클러스터 상세 정보:")
    subprocess.run(["az", "aks", "show", "--resource-group", RESOURCE_GROUP, 
                    "--name", AKS_CLUSTER_NAME, "--output", "table"])
else:
    print("❌ 클러스터 생성 실패")

### 2.4 클러스터 인증 정보 가져오기

kubectl을 사용하기 위해 AKS 클러스터의 인증 정보를 가져옵니다.

> **GitHub Codespaces**: kubectl이 이미 설치되어 있습니다.

In [ ]:
import subprocess

# 위 셀에서 설정한 변수 사용
print(f"🔑 클러스터 인증 정보 가져오기: {AKS_CLUSTER_NAME}")
print()

# 클러스터 인증 정보 가져오기
subprocess.run(["az", "aks", "get-credentials", 
                "--resource-group", RESOURCE_GROUP, 
                "--name", AKS_CLUSTER_NAME])

print()
print("📊 kubectl 연결 확인:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("ℹ️ 클러스터 정보:")
subprocess.run(["kubectl", "cluster-info"])

## 3. 컨테이너 이미지 AKS에 배포하기

앞서 ACR에 푸시한 이미지를 AKS에 배포합니다.

### 3.1 ACR을 AKS에 연결

AKS 클러스터가 ACR에 접근할 수 있도록 권한을 부여합니다.

In [ ]:
import subprocess

# config.py에서 가져온 ACR_NAME 사용
print("🔍 ACR 정보 확인 중...")
print(f"   config.py에서 가져온 ACR 이름: {ACR_NAME}")
print()

if ACR_NAME is None:
    # config.py에 ACR_NAME이 설정되지 않은 경우, 리소스 그룹에서 자동 조회
    print("⚠️  config.py에 ACR_NAME이 설정되지 않았습니다.")
    print("   리소스 그룹에서 ACR을 자동으로 조회합니다...")
    print()
    acr_result = subprocess.run(
        ["az", "acr", "list", "--resource-group", RESOURCE_GROUP, "--query", "[0].name", "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if acr_result.returncode == 0 and acr_result.stdout.strip():
        ACR_NAME = acr_result.stdout.strip()
        print(f"✅ ACR 발견: {ACR_NAME}")
        print()
        print("💡 권장 사항:")
        print(f"   config.py 파일을 열고 ACR_NAME = \"{ACR_NAME}\"로 업데이트하세요.")
    else:
        print("❌ ACR을 찾을 수 없습니다. 1번 노트북을 먼저 실행해주세요.")
        raise Exception("ACR not found")
else:
    print(f"✅ ACR 이름: {ACR_NAME}")

print()
print(f"🔗 ACR을 AKS에 연결 중...")
print(f"   ACR: {ACR_NAME}")
print(f"   AKS: {AKS_CLUSTER_NAME}")
print()

# ACR을 AKS에 연결 (attach-acr)
# 이 명령이 자동으로 AcrPull 권한을 kubelet identity에 부여합니다

result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--attach-acr", ACR_NAME
])

if result.returncode == 0:
    print("✅ ACR 연결 완료!")
    print()
    print("   AKS가 ACR에서 이미지를 pull할 수 있는 권한이 자동으로 설정되었습니다.")
    print()
    
    # 클러스터가 ACR 권한을 인식할 때까지 대기
    print("⏱️  클러스터가 ACR 권한을 인식할 때까지 30초 대기 중...")
    import time
    for i in range(30, 0, -5):
        print(f"   {i}초 남음...", end='\r')
        time.sleep(5)
    print("   ✓ 대기 완료!" + " " * 20)
    print()
    
    # 클러스터 상태 확인
    print("📊 클러스터 상태 확인:")
    subprocess.run(["az", "aks", "show", 
                    "--resource-group", RESOURCE_GROUP, 
                    "--name", AKS_CLUSTER_NAME,
                    "--query", "{provisioningState:provisioningState, powerState:powerState.code, nodeResourceGroup:nodeResourceGroup}",
                    "--output", "table"])
    print()
    
    # ACR 연결 상태 확인
    print("🔍 ACR 연결 상태 확인:")
    acr_id_result = subprocess.run(
        ["az", "acr", "show", "--name", ACR_NAME, "--query", "id", "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if acr_id_result.returncode == 0:
        acr_id = acr_id_result.stdout.strip()
        print(f"   ACR ID: {acr_id}")
        
        # kubelet identity의 역할 할당 확인
        print("   Kubelet identity의 AcrPull 권한 확인 중...")
        role_result = subprocess.run(
            ["az", "role", "assignment", "list",
             "--scope", acr_id,
             "--query", "[?roleDefinitionName=='AcrPull'].{Principal:principalName, Role:roleDefinitionName, Scope:scope}",
             "--output", "table"],
            capture_output=True,
            text=True
        )
        
        if role_result.returncode == 0 and role_result.stdout.strip():
            print("   ✅ AcrPull 권한이 정상적으로 할당되었습니다.")
            print()
            print(role_result.stdout)
        else:
            print("   ⚠️  권한 확인 중 오류 발생 (하지만 정상적으로 작동할 수 있습니다)")
    print()
else:
    print("❌ ACR 연결 실패")

print()

### 3.2 ACR 이미지 확인

배포하기 전에 ACR에 어떤 이미지가 있는지 확인합니다.

In [ ]:
import subprocess

print(f"📦 ACR 저장소 목록 확인: {ACR_NAME}")
print()

# ACR 저장소 목록 조회
repo_result = subprocess.run(
    ["az", "acr", "repository", "list", "--name", ACR_NAME, "--output", "table"],
    capture_output=True,
    text=True
)

if repo_result.returncode == 0:
    print(repo_result.stdout)
    
    # 저장소 이름 추출하여 태그 확인
    repos_output = subprocess.run(
        ["az", "acr", "repository", "list", "--name", ACR_NAME, "-o", "tsv"],
        capture_output=True,
        text=True
    )
    
    if repos_output.returncode == 0 and repos_output.stdout.strip():
        repos = repos_output.stdout.strip().split('\n')
        print()
        print("🏷️  각 이미지의 태그 확인:")
        print()
        
        for repo in repos:
            if repo:
                print(f"   📂 저장소: {repo}")
                subprocess.run([
                    "az", "acr", "repository", "show-tags",
                    "--name", ACR_NAME,
                    "--repository", repo,
                    "--output", "table"
                ])
                print()
else:
    print("❌ ACR 저장소 목록 조회 실패")
    print(repo_result.stderr)

print()
print("💡 1번 노트북에서 이미지를 푸시했는지 확인하세요.")
print("   이미지가 없다면 1번 노트북의 섹션 6.7(이미지 푸시)를 실행하세요.")

### 3.3 Deployment YAML 파일 작성

`deployment.yaml` 파일을 자동으로 생성합니다.

In [ ]:
# config.py에서 가져온 변수 사용하여 deployment.yaml 생성
deployment_yaml = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: {DEPLOYMENT_NAME}
spec:
  replicas: {REPLICAS}
  selector:
    matchLabels:
      app: {APP_NAME}
  template:
    metadata:
      labels:
        app: {APP_NAME}
    spec:
      containers:
      - name: {APP_NAME}
        image: {get_full_image_name(ACR_NAME, APP_NAME, IMAGE_TAG)}
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi
"""

# 파일 생성
with open("deployment.yaml", "w") as f:
    f.write(deployment_yaml)

print(f"✅ deployment.yaml 파일 생성 완료")
print(f"   이미지: {get_full_image_name(ACR_NAME, APP_NAME, IMAGE_TAG)}")
print()
print("💡 사용된 config.py 변수:")
print(f"   DEPLOYMENT_NAME: {DEPLOYMENT_NAME}")
print(f"   APP_NAME: {APP_NAME}")
print(f"   REPLICAS: {REPLICAS}")
print(f"   ACR_NAME: {ACR_NAME}")
print()
print("📄 생성된 파일 내용:")
print()
print(deployment_yaml)

### 3.4 Deployment 생성 및 Service 노출

In [ ]:
import subprocess

print("📦 Deployment 생성 중...")
subprocess.run(["kubectl", "apply", "-f", "deployment.yaml"])

print()
print("📊 Deployment 상태 확인:")
subprocess.run(["kubectl", "get", "deployments"])

print()
print("🔍 Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("🌐 Service 생성 중 (LoadBalancer)...")
subprocess.run(["kubectl", "expose", "deployment", DEPLOYMENT_NAME, 
                "--type=LoadBalancer", 
                "--port=80", 
                "--target-port=8080"])

print()
print("📋 Service 상태 확인:")
subprocess.run(["kubectl", "get", "services"])

### 3.5 외부 IP 확인 및 애플리케이션 접근

LoadBalancer 서비스가 생성되면 Azure가 자동으로 공용 IP를 할당합니다. 이 IP를 통해 애플리케이션에 접근할 수 있습니다.

> **참고**: 외부 IP 할당에는 1-2분 정도 소요될 수 있습니다. `<pending>` 상태에서 실제 IP 주소로 변경될 때까지 기다려주세요.

In [ ]:
import subprocess

print("🌐 외부 IP 확인")
print("=" * 80)
print()

print(f"📋 Service 정보 확인: {SERVICE_NAME}")
subprocess.run(["kubectl", "get", "service", SERVICE_NAME])

print()

print("📊 외부 IP 상세 정보:")
result = subprocess.run([
    "kubectl", "get", "service", SERVICE_NAME,
    "-o", "jsonpath={.status.loadBalancer.ingress[0].ip}"
], capture_output=True, text=True)

external_ip = result.stdout.strip()

if external_ip:
    print(f"✅ 외부 IP: {external_ip}")
    print()
    print("🌐 애플리케이션 접근:")
    print(f"   브라우저: http://{external_ip}")
    print(f"   또는 터미널: curl http://{external_ip}")
    print()
    print("💡 애플리케이션이 정상적으로 응답하는지 확인하세요!")
else:
    print("⚠️  외부 IP가 아직 할당되지 않았습니다 (<pending> 상태)")
    print()
    print("💡 다음 단계:")
    print("   1. 1-2분 후 이 셀을 다시 실행하세요")
    print("   2. 또는 다음 명령으로 실시간 모니터링:")
    print(f"      kubectl get service {SERVICE_NAME} --watch")

print()
print("=" * 80)

## 4. 애플리케이션 확장

Kubernetes에서는 replica 수를 조정하여 애플리케이션을 쉽게 확장할 수 있습니다.

### 4.1 수동 스케일링

In [ ]:
import subprocess

print("📈 Replica 수를 4로 확장 중...")
subprocess.run(["kubectl", "scale", "deployment", "myapp", "--replicas=4"])

print()
print("🔍 Pod 상태 확인 (상세):")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("📊 Deployment 상태:")
subprocess.run(["kubectl", "get", "deployment", "myapp"])

### 4.2 HPA(Horizontal Pod Autoscaler)로 오토스케일링 적용

HPA를 사용하면 CPU 사용률 등의 메트릭에 따라 자동으로 Pod 수를 조정할 수 있습니다.

In [ ]:
import subprocess

print("🎯 HPA 생성 중 (CPU 사용률 50% 기준, 최소 2개, 최대 10개)...")
subprocess.run(["kubectl", "autoscale", "deployment", "myapp", 
                "--cpu-percent=50", 
                "--min=2", 
                "--max=10"])

print()
print("📊 HPA 상태:")
subprocess.run(["kubectl", "get", "hpa"])

print()
print("📋 HPA 상세 정보:")
subprocess.run(["kubectl", "describe", "hpa", "myapp"])

### 4.3 HPA 오토스케일링 테스트

HPA가 정상적으로 동작하는지 확인하기 위해 부하를 발생시킵니다.

#### 방법 1: 부하 생성 도구 사용

In [ ]:
import subprocess

print("🔥 부하 생성용 Pod 실행 중...")
subprocess.run(["kubectl", "run", "load-generator", 
                "--image=busybox", 
                "--restart=Never", 
                "--", "/bin/sh", "-c", 
                "while true; do wget -q -O- http://myapp; done"])

print()
print("✅ 부하 생성 Pod가 실행되었습니다.")
print()
print("   HPA 상태를 모니터링하려면 별도 터미널에서 다음 명령 실행:")
print("   kubectl get hpa myapp --watch")
print()
print("   또는 Pod 수 변화 모니터링:")
print("   kubectl get pods --watch")

#### 테스트 정리

부하 테스트가 완료되면 부하 생성 Pod를 삭제합니다.

In [ ]:
import subprocess

print("🧹 부하 생성 Pod 삭제 중...")
subprocess.run(["kubectl", "delete", "pod", "load-generator"])

print()
print("✅ 부하 생성 Pod가 삭제되었습니다.")

### HPA 오토스케일링 테스트 방법
HPA가 정상적으로 동작하는지 확인하려면, 파드에 부하를 인위적으로 발생시켜 오토스케일링이 동작하는지 관찰할 수 있습니다.

1. **stress 도구 설치** (app 파드에 설치):
```bash
kubectl exec -it <app-pod-name> -- apt-get update
kubectl exec -it <app-pod-name> -- apt-get install -y stress
```

2. **CPU 부하 발생**:
```bash
kubectl exec -it <app-pod-name> -- stress --cpu 1 --timeout 300
```

3. **HPA 상태 모니터링**:
```bash
kubectl get hpa -w
```

- 위 과정을 통해 CPU 사용률이 증가하면 HPA가 자동으로 파드 수를 늘리는 것을 확인할 수 있습니다.

 > 참고: `<app-pod-name>`은 실제 app 파드 이름으로 대체하세요. 파드 이름은 `kubectl get pods` 명령으로 확인할 수 있습니다.

## 5. 모니터링 및 관찰성(Observability)

AKS는 Azure Monitor, Prometheus, Grafana, Container Insights와 통합되어 클러스터 및 애플리케이션의 완전한 관찰성을 제공합니다.

**관찰성의 3가지 핵심 요소:**
1. **메트릭(Metrics)**: Prometheus로 실시간 성능 지표 수집
2. **로그(Logs)**: Container Insights로 컨테이너 로그 및 이벤트 수집
3. **시각화(Visualization)**: Grafana로 대시보드를 통한 데이터 시각화

### 5.1 Azure Monitor 기본 모니터링

Azure Portal에서 AKS 리소스를 선택하고, 모니터링 탭에서 다음을 확인할 수 있습니다:
- **Insights**: 클러스터, 노드, 컨트롤러, 컨테이너 상태
- **Metrics**: CPU, Memory, Network 등의 메트릭
- **Logs**: KQL(Kusto Query Language)로 로그 쿼리
- **Workbooks**: 사전 구성된 모니터링 대시보드

### 5.2 관리형 Prometheus, Grafana 및 Container Insights 활성화

AKS 클러스터에 완전한 관찰성(Observability) 스택을 구성합니다:
- **Azure Monitor Managed Prometheus**: 메트릭 수집 (CPU, Memory, Network 등)
- **Azure Managed Grafana**: 시각화 대시보드
- **Container Insights**: 컨테이너 로그 및 이벤트 수집

이 3가지 솔루션을 통해 메트릭, 로그, 시각화를 모두 제공하는 통합 모니터링 환경을 구축합니다.

In [ ]:
import subprocess
import warnings

# Azure CLI 확장의 pkg_resources 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='.*pkg_resources.*')

print("🔧 관리형 Prometheus, Grafana 및 Container Insights 활성화 중...")
print("=" * 80)
print()
print("📊 구성할 모니터링 스택:")
print("   1. Azure Monitor Workspace (Prometheus 메트릭)")
print("   2. AKS Prometheus Addon (메트릭 수집 에이전트)")
print("   3. Azure Managed Grafana (시각화)")
print("   4. Log Analytics Workspace (컨테이너 로그)")
print("   5. Container Insights (로그 수집 에이전트)")
print("=" * 80)
print()

# config.py에서 가져온 변수 사용
# Azure Monitor 워크스페이스 생성
print("1️⃣ Azure Monitor 워크스페이스 생성")
print(f"   워크스페이스 이름: {MONITOR_WORKSPACE_NAME}")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print(f"   위치: {LOCATION}")
print()

result = subprocess.run([
    "az", "monitor", "account", "create",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--location", LOCATION
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Azure Monitor 워크스페이스 생성 완료: {MONITOR_WORKSPACE_NAME}")
else:
    print("⚠️ 워크스페이스가 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

print()

# Monitor 워크스페이스 ID 가져오기
print("2️⃣ 워크스페이스 ID 확인")
ws_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id",
    "-o", "tsv"
], capture_output=True, text=True)

if ws_result.returncode == 0:
    MONITOR_WORKSPACE_ID = ws_result.stdout.strip()
    print(f"✅ 워크스페이스 ID: {MONITOR_WORKSPACE_ID}")
else:
    print("❌ 워크스페이스 ID 조회 실패")
    print(ws_result.stderr)

print()

# AKS에 Prometheus 애드온 활성화
print("3️⃣ AKS에 Prometheus 애드온 활성화")
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--enable-azure-monitor-metrics",
    "--azure-monitor-workspace-resource-id", MONITOR_WORKSPACE_ID
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Prometheus 애드온 활성화 완료")
else:
    print("⚠️ Prometheus 애드온 활성화 중 오류")
    print(result.stderr)

print()

# config.py에서 가져온 변수 사용
# Grafana 워크스페이스 생성
print("4️⃣ 관리형 Grafana 워크스페이스 생성")
print(f"   Grafana 이름: {GRAFANA_NAME}")
print(f"   리소스 그룹: {RESOURCE_GROUP}")
print()

result = subprocess.run([
    "az", "grafana", "create",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Grafana 워크스페이스 생성 완료: {GRAFANA_NAME}")
else:
    print("⚠️ Grafana가 이미 존재하거나 생성 중 오류 발생")
    print(result.stderr)

print()

# Grafana와 Prometheus 연결
print("5️⃣ Grafana와 Prometheus 데이터 소스 연결")
grafana_result = subprocess.run([
    "az", "grafana", "show",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "id",
    "-o", "tsv"
], capture_output=True, text=True)

if grafana_result.returncode == 0:
    GRAFANA_ID = grafana_result.stdout.strip()
    
    # Prometheus를 Grafana 데이터 소스로 추가
    subprocess.run([
        "az", "aks", "update",
        "--resource-group", RESOURCE_GROUP,
        "--name", AKS_CLUSTER_NAME,
        "--grafana-resource-id", GRAFANA_ID
    ])
    
    print("✅ Grafana와 Prometheus 연결 완료")
else:
    print("⚠️ Grafana 연결 중 오류")

print()

# Container Insights (로그 수집) 활성화
print("6️⃣ Container Insights 활성화 (선택사항)")
print("   컨테이너 로그 및 이벤트 수집을 위한 Log Analytics 워크스페이스 연결")
print()

# Log Analytics 워크스페이스 이름 설정
LOG_WORKSPACE_NAME = f"{AKS_CLUSTER_NAME}-logs"

# Log Analytics 워크스페이스 생성 (또는 기존 것 사용)
print(f"   Log Analytics 워크스페이스 생성 중: {LOG_WORKSPACE_NAME}")
log_ws_result = subprocess.run([
    "az", "monitor", "log-analytics", "workspace", "create",
    "--resource-group", RESOURCE_GROUP,
    "--workspace-name", LOG_WORKSPACE_NAME,
    "--location", LOCATION
], capture_output=True, text=True)

if log_ws_result.returncode == 0 or "already exists" in log_ws_result.stderr.lower():
    if log_ws_result.returncode == 0:
        print("   ✅ Log Analytics 워크스페이스 생성 완료")
    else:
        print("   ✅ Log Analytics 워크스페이스가 이미 존재합니다")
    
    # Log Analytics 워크스페이스 ID 가져오기
    log_ws_id_result = subprocess.run([
        "az", "monitor", "log-analytics", "workspace", "show",
        "--resource-group", RESOURCE_GROUP,
        "--workspace-name", LOG_WORKSPACE_NAME,
        "--query", "id",
        "-o", "tsv"
    ], capture_output=True, text=True)
    
    if log_ws_id_result.returncode == 0:
        LOG_WS_ID = log_ws_id_result.stdout.strip()
        print(f"   ✅ 워크스페이스 ID: {LOG_WS_ID}")
        print()
        
        # Container Insights 활성화
        print("   Container Insights addon 활성화 중...")
        insights_result = subprocess.run([
            "az", "aks", "enable-addons",
            "--addon", "monitoring",
            "--resource-group", RESOURCE_GROUP,
            "--name", AKS_CLUSTER_NAME,
            "--workspace-resource-id", LOG_WS_ID
        ], capture_output=True, text=True)
        
        if insights_result.returncode == 0:
            print("   ✅ Container Insights 활성화 완료")
        else:
            print("   ⚠️ Container Insights가 이미 활성화되어 있거나 오류 발생")
            print(f"   {insights_result.stderr}")
    else:
        print("   ❌ 워크스페이스 ID 조회 실패")
else:
    print("   ⚠️ Log Analytics 워크스페이스 생성 중 오류")
    print(f"   {log_ws_result.stderr}")

print()

print("🎉 모니터링 스택 설정 완료!")
print("=" * 80)
print()
print("📊 생성된 리소스:")
print(f"   - Azure Monitor 워크스페이스: {MONITOR_WORKSPACE_NAME} (Prometheus 메트릭)")
print(f"   - Grafana 워크스페이스: {GRAFANA_NAME} (시각화)")
print(f"   - Log Analytics 워크스페이스: {LOG_WORKSPACE_NAME} (컨테이너 로그)")
print()
print("💡 다음 단계:")
print("   1. Azure Portal에서 Grafana 워크스페이스 접속")
print("   2. Prometheus 데이터 소스 확인")
print("   3. Kubernetes 표준 대시보드 활용")
print("   4. Log Analytics에서 컨테이너 로그 쿼리 (KQL)")

### 5.3 Grafana 접속 URL 확인

In [ ]:
import subprocess

print("🔗 Grafana 대시보드 URL 확인:")
print()

# Grafana 이름 확인
try:
    print(f"📝 사용 중인 Grafana 이름: {GRAFANA_NAME}")
    print(f"📝 리소스 그룹: {RESOURCE_GROUP}")
    print()
except NameError:
    print("⚠️  GRAFANA_NAME 변수가 설정되지 않았습니다.")
    print("   섹션 5.2를 먼저 실행하여 Grafana를 생성하세요.")
    raise

# Grafana 존재 여부 먼저 확인
print("🔍 Grafana 워크스페이스 존재 확인 중...")
check_result = subprocess.run([
    "az", "grafana", "list",
    "--resource-group", RESOURCE_GROUP,
    "--query", "[?name=='" + GRAFANA_NAME + "'].name",
    "-o", "tsv"
], capture_output=True, text=True)

if check_result.returncode == 0 and check_result.stdout.strip():
    print(f"✅ Grafana 워크스페이스 발견: {check_result.stdout.strip()}")
    print()
    
    # URL 조회
    result = subprocess.run([
        "az", "grafana", "show",
        "--name", GRAFANA_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--query", "properties.endpoint",
        "-o", "tsv"
    ], capture_output=True, text=True)

    if result.returncode == 0 and result.stdout.strip():
        grafana_url = result.stdout.strip()
        print(f"✅ Grafana URL: {grafana_url}")
        print()
        print("💡 브라우저에서 위 URL로 접속하여 Grafana 대시보드를 확인하세요.")
        print("   Azure AD 계정으로 자동 로그인됩니다.")
    else:
        print("❌ Grafana URL 조회 실패")
        print(f"   에러: {result.stderr}")
else:
    print(f"❌ Grafana 워크스페이스를 찾을 수 없습니다: {GRAFANA_NAME}")
    print()
    print("💡 해결 방법:")
    print("   1. 섹션 5.2를 다시 실행하여 Grafana를 생성하세요.")
    print("   2. 또는 기존 Grafana 목록을 확인하세요:")
    print()
    
    # 리소스 그룹의 모든 Grafana 목록 출력
    list_result = subprocess.run([
        "az", "grafana", "list",
        "--resource-group", RESOURCE_GROUP,
        "--output", "table"
    ])

### 5.4 Grafana Prometheus 데이터 소스 확인 및 추가

Grafana에 Prometheus 데이터 소스가 제대로 설정되어 있는지 확인하고, 없으면 자동으로 추가합니다.

In [ ]:
import subprocess
import json

print("🔍 Grafana Prometheus 데이터 소스 확인 및 추가")
print("=" * 80)
print()

# 1. Azure Monitor 워크스페이스 정보 가져오기
print("1️⃣ Azure Monitor 워크스페이스 정보 확인:")
monitor_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "{id:id, endpoint:metrics.prometheusQueryEndpoint}",
    "-o", "json"
], capture_output=True, text=True)

if monitor_result.returncode == 0:
    try:
        monitor_info = json.loads(monitor_result.stdout)
        monitor_workspace_id = monitor_info['id']
        prometheus_endpoint = monitor_info.get('endpoint', f"https://{MONITOR_WORKSPACE_NAME}.prometheus.monitor.azure.com")
        print(f"✅ Monitor Workspace ID: {monitor_workspace_id}")
        print(f"✅ Prometheus Endpoint: {prometheus_endpoint}")
    except:
        print("⚠️  Monitor 워크스페이스 정보 파싱 실패")
        monitor_workspace_id = None
        prometheus_endpoint = None
else:
    print("❌ Monitor 워크스페이스를 찾을 수 없습니다.")
    monitor_workspace_id = None
    prometheus_endpoint = None

print()

# 2. 현재 Grafana 데이터 소스 목록 확인
print("2️⃣ 현재 Grafana 데이터 소스 확인:")
ds_list_result = subprocess.run([
    "az", "grafana", "data-source", "list",
    "--name", GRAFANA_NAME,
    "--resource-group", RESOURCE_GROUP,
    "-o", "json"
], capture_output=True, text=True)

prometheus_ds_exists = False
if ds_list_result.returncode == 0:
    try:
        data_sources = json.loads(ds_list_result.stdout)
        print(f"   총 {len(data_sources)}개의 데이터 소스 발견:")
        for ds in data_sources:
            print(f"   - {ds['name']} (타입: {ds['type']})")
            if "Azure Monitor managed service for Prometheus" in ds['name']:
                prometheus_ds_exists = True
                print("     ✅ Prometheus 데이터 소스 이미 존재!")
    except:
        print("   ⚠️  데이터 소스 목록 파싱 실패")
else:
    print("   ❌ 데이터 소스 목록 조회 실패")

print()

# 3. Prometheus 데이터 소스가 없으면 추가
if not prometheus_ds_exists and monitor_workspace_id and prometheus_endpoint:
    print("3️⃣ Prometheus 데이터 소스 추가 중...")
    print()
    
    # 구독 ID 추출
    subscription_id = monitor_workspace_id.split('/')[2]
    
    # 데이터 소스 정의
    datasource_def = {
        "access": "proxy",
        "jsonData": {
            "azureMonitorWorkspaceResourceId": monitor_workspace_id,
            "subscriptionId": subscription_id
        },
        "name": "Azure Monitor managed service for Prometheus",
        "type": "prometheus",
        "url": prometheus_endpoint
    }
    
    # 데이터 소스 추가
    ds_create_result = subprocess.run([
        "az", "grafana", "data-source", "create",
        "--name", GRAFANA_NAME,
        "--resource-group", RESOURCE_GROUP,
        "--definition", json.dumps(datasource_def)
    ], capture_output=True, text=True)
    
    if ds_create_result.returncode == 0:
        print("✅ Prometheus 데이터 소스 추가 완료!")
        try:
            result = json.loads(ds_create_result.stdout)
            print(f"   데이터 소스 ID: {result.get('id')}")
            print(f"   데이터 소스 이름: {result.get('name')}")
        except:
            pass
    else:
        print("❌ 데이터 소스 추가 실패:")
        print(ds_create_result.stderr)
elif prometheus_ds_exists:
    print("3️⃣ Prometheus 데이터 소스가 이미 존재합니다.")
    print("   ✅ 설정이 완료되어 있습니다!")
else:
    print("3️⃣ Monitor 워크스페이스 정보가 없어 데이터 소스를 추가할 수 없습니다.")
    print("   섹션 5.2를 먼저 실행하세요.")

print()
print("=" * 80)
print("💡 확인 방법:")
print("   1. Grafana URL로 접속 (섹션 5.3 참조)")
print("   2. 좌측 메뉴 > Connections > Data sources 확인")
print("   3. 'Azure Monitor managed service for Prometheus' 데이터 소스 확인")
print("   4. 좌측 메뉴 > Dashboards > Browse에서 Kubernetes 대시보드 확인")
print("=" * 80)

### 5.5 Prometheus 데이터 수집 확인

Prometheus가 AKS 클러스터에서 메트릭을 정상적으로 수집하고 있는지 확인합니다.

In [ ]:
import subprocess
import json

print("🔍 Prometheus 데이터 수집 상태 확인")
print("=" * 80)
print()

# 1. Azure Monitor Workspace 정보 확인
print("1️⃣ Azure Monitor Workspace 확인:")
monitor_result = subprocess.run([
    "az", "monitor", "account", "show",
    "--name", MONITOR_WORKSPACE_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "{id:id, endpoint:metrics.prometheusQueryEndpoint}",
    "-o", "json"
], capture_output=True, text=True)

if monitor_result.returncode == 0:
    monitor_info = json.loads(monitor_result.stdout)
    print(f"   ✅ 워크스페이스: {MONITOR_WORKSPACE_NAME}")
    print(f"   📊 Query Endpoint: {monitor_info.get('endpoint', 'N/A')}")
    print()
else:
    print("   ❌ Azure Monitor Workspace를 찾을 수 없습니다.")
    print()

# 2. AKS 클러스터의 Prometheus addon 상태 확인
print("2️⃣ AKS Prometheus Addon 상태 확인:")
addon_result = subprocess.run([
    "az", "aks", "show",
    "--name", AKS_CLUSTER_NAME,
    "--resource-group", RESOURCE_GROUP,
    "--query", "azureMonitorProfile.metrics",
    "-o", "json"
], capture_output=True, text=True)

if addon_result.returncode == 0:
    try:
        metrics_info = json.loads(addon_result.stdout)
        if metrics_info and metrics_info.get('enabled'):
            print(f"   ✅ Prometheus Metrics Addon: Enabled")
        else:
            print("   ⚠️  Prometheus Metrics Addon이 활성화되지 않았습니다.")
            print("   섹션 5.2를 다시 확인하세요.")
    except:
        print("   ⚠️  Metrics 정보를 파싱할 수 없습니다.")
else:
    print("   ❌ 클러스터 정보를 가져올 수 없습니다.")
print()

# 3. Prometheus 수집 Pod 상태 확인
print("3️⃣ Prometheus 데이터 수집 Pod 상태 확인:")
print("   (ama-metrics DaemonSet 및 Deployment)")
print()

# ama-metrics-node DaemonSet (Linux nodes)
print("   📦 ama-metrics-node DaemonSet (Linux):")
subprocess.run([
    "kubectl", "get", "daemonset", "ama-metrics-node",
    "-n", "kube-system",
    "-o", "wide"
])
print()

# ama-metrics Deployment
print("   📦 ama-metrics Deployment (Scraper):")
subprocess.run([
    "kubectl", "get", "deployment", "ama-metrics",
    "-n", "kube-system",
    "-o", "wide"
])
print()

# ama-metrics-ksm Deployment
print("   📦 ama-metrics-ksm Deployment (Kube State Metrics):")
subprocess.run([
    "kubectl", "get", "deployment", "ama-metrics-ksm",
    "-n", "kube-system",
    "-o", "wide"
])
print()

# 4. Pod 로그 확인 (최근 오류 체크)
print("4️⃣ 데이터 수집 Pod 로그 확인 (최근 10줄):")
print()

# ama-metrics pod 찾기
pod_result = subprocess.run([
    "kubectl", "get", "pods",
    "-n", "kube-system",
    "-l", "rsName=ama-metrics",
    "-o", "jsonpath={.items[0].metadata.name}"
], capture_output=True, text=True)

if pod_result.returncode == 0 and pod_result.stdout.strip():
    pod_name = pod_result.stdout.strip()
    print(f"   Pod: {pod_name}")
    print("   최근 로그:")
    print("   " + "-" * 70)
    
    log_result = subprocess.run([
        "kubectl", "logs",
        pod_name,
        "-n", "kube-system",
        "--tail=10"
    ], capture_output=True, text=True)
    
    if log_result.returncode == 0:
        for line in log_result.stdout.split('\n')[:10]:
            if line.strip():
                print(f"   {line}")
    else:
        print("   로그를 가져올 수 없습니다.")
    print("   " + "-" * 70)
else:
    print("   ⚠️  ama-metrics Pod를 찾을 수 없습니다.")

print()
print("=" * 80)
print("✅ 정상 상태 확인 사항:")
print("   1. ama-metrics-node DaemonSet: READY = DESIRED (클러스터 노드 수와 일치)")
print("   2. ama-metrics Deployment: READY = 1/1")
print("   3. ama-metrics-ksm Deployment: READY = 1/1")
print("   4. Pod 로그에 'error', 'failed', 'connection refused' 메시지가 없어야 함")
print()
print("💡 데이터 수집이 정상이라면:")
print("   - Azure Monitor Workspace에 메트릭이 저장되고 있습니다.")
print("   - Grafana에서 5-10분 후 메트릭이 표시됩니다.")
print("   - 다음 섹션(5.6)에서 대시보드를 가져와 확인하세요.")
print("=" * 80)

### 5.6 Grafana 대시보드 가져오기

Grafana에 Kubernetes 모니터링용 오픈소스 대시보드를 가져와서 클러스터를 시각화합니다.

#### 📊 추천 오픈소스 Kubernetes 대시보드

Grafana.com에서 제공하는 검증된 Kubernetes 모니터링 대시보드를 사용할 수 있습니다:

| Dashboard ID | 이름 | 설명 | 인기도 |
|--------------|------|------|--------|
| **15760** | Kubernetes / Views / Global | 클러스터 전체 개요 (노드, 파드, 네임스페이스) | ⭐⭐⭐⭐⭐ |
| **15761** | Kubernetes / Views / Namespaces | 네임스페이스별 리소스 사용량 | ⭐⭐⭐⭐⭐ |
| **15762** | Kubernetes / Views / Nodes | 노드별 상세 메트릭 | ⭐⭐⭐⭐⭐ |
| **15763** | Kubernetes / Views / Pods | 파드별 CPU, Memory, Network | ⭐⭐⭐⭐⭐ |
| **315** | Kubernetes cluster monitoring (via Prometheus) | 클래식 클러스터 모니터링 | ⭐⭐⭐⭐ |
| **6417** | Kubernetes Cluster (Prometheus) | 상세 클러스터 대시보드 | ⭐⭐⭐⭐ |
| **7249** | Kubernetes Cluster Overview | 간단한 클러스터 개요 | ⭐⭐⭐ |

> **💡 권장**: **15760~15763** 대시보드 세트는 Kubernetes 커뮤니티에서 가장 많이 사용되는 표준 대시보드입니다.

#### 방법 1: Grafana Portal에서 수동으로 가져오기 (권장)

1. **Grafana 접속**
   - Azure Portal → Grafana 리소스 → "Endpoint" URL 클릭
   - Azure AD로 자동 로그인

2. **대시보드 Import**
   - 좌측 메뉴 → **Dashboards** → **New** → **Import**
   
3. **Dashboard ID 입력**
   - 위 표에서 원하는 Dashboard ID 입력 (예: `15760`)
   - **Load** 버튼 클릭

4. **데이터 소스 선택**
   - Prometheus 드롭다운에서 **"Azure Monitor managed service for Prometheus"** 선택
   - **Import** 버튼 클릭

5. **반복**
   - 다른 대시보드도 동일한 방법으로 추가 (15761, 15762, 15763 등)

#### 💡 자동 설치된 대시보드

Azure Managed Grafana를 AKS와 연결하면 (`--grafana-resource-id`) **Kubernetes 표준 대시보드가 자동으로 설치**됩니다!

섹션 5.2를 실행한 후 Grafana에 접속하면 다음 대시보드들이 이미 준비되어 있습니다:
- **Kubernetes / Views / Global**: 클러스터 전체 개요
- **Kubernetes / Views / Namespaces**: 네임스페이스별 리소스
- **Kubernetes / Views / Nodes**: 노드별 상세 메트릭
- **Kubernetes / Views / Pods**: 파드별 리소스 사용량

> **참고**: 추가 대시보드가 필요한 경우 방법 1(수동 Import)을 사용하세요.

## 6. Node Auto Provisioning(노드 자동 프로비저닝) 적용

Node Auto Provisioning(NAP)은 AKS에서 워크로드 수요에 따라 다양한 VM 크기의 노드 풀을 자동으로 생성/확장/축소하는 기능입니다. 비용 최적화와 리소스 효율성을 위해 권장되는 Azure 기능입니다.

### 6.1 사전 준비

NAP를 사용하기 위해서는 먼저 필요한 Azure CLI 확장을 설치하고 기능 플래그를 등록해야 합니다.

In [ ]:
import subprocess

print("📦 aks-preview 확장 설치/업데이트 중...")
print()

# aks-preview 확장 설치 또는 업데이트
result = subprocess.run([
    "az", "extension", "add",
    "--name", "aks-preview",
    "--yes"
], capture_output=True, text=True)

if "already installed" in result.stderr.lower() or result.returncode == 0:
    print("✅ aks-preview 확장이 설치되어 있습니다. 최신 버전으로 업데이트 중...")
    subprocess.run([
        "az", "extension", "update",
        "--name", "aks-preview"
    ])
else:
    print("✅ aks-preview 확장 설치 완료")

print()
print("🔧 NodeAutoProvisioningPreview Feature Flag 등록 중...")
print()

# Feature Flag 등록
subprocess.run([
    "az", "feature", "register",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview"
])

print()
print("📋 Feature Flag 등록 상태 확인:")
subprocess.run([
    "az", "feature", "show",
    "--namespace", "Microsoft.ContainerService",
    "--name", "NodeAutoProvisioningPreview",
    "--output", "table"
])

print()
print("💡 참고:")
print("   - Feature Flag가 'Registered' 상태가 될 때까지 몇 분이 걸릴 수 있습니다.")
print("   - 'Registered' 상태 확인 후 다음 셀을 실행하세요.")

### 6.2 Provider 재등록

Feature Flag가 'Registered' 상태가 되면 Provider를 재등록해야 합니다.

In [ ]:
import subprocess

print("🔄 Microsoft.ContainerService Provider 재등록 중...")
print()

result = subprocess.run([
    "az", "provider", "register",
    "--namespace", "Microsoft.ContainerService"
])

if result.returncode == 0:
    print("✅ Provider 재등록 완료")
    print()
    print("💡 다음 단계:")
    print("   몇 분 후 섹션 6.3을 실행하여 NAP를 활성화하세요.")
else:
    print("❌ Provider 재등록 실패")

### 6.3 기존 클러스터에 NAP 활성화

기존 AKS 클러스터에 Node Auto Provisioning을 활성화합니다.

> **중요**: NAP를 활성화하면 기존 수동 노드 풀은 유지되지만, 향후 자동 프로비저닝된 노드 풀이 추가로 생성됩니다.

In [ ]:
import subprocess

print(f"🚀 기존 클러스터에 NAP 활성화 중: {AKS_CLUSTER_NAME}")
print("⏱️  예상 소요 시간: 5-10분")
print()

# NAP 활성화
result = subprocess.run([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", AKS_CLUSTER_NAME,
    "--node-provisioning-mode", "Auto"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ NAP 활성화 완료!")
    print()
    print("📊 클러스터 상태 확인:")
    subprocess.run([
        "az", "aks", "show",
        "--resource-group", RESOURCE_GROUP,
        "--name", AKS_CLUSTER_NAME,
        "--query", "{Name:name, NodeProvisioningMode:nodeProvisioningProfile.mode}",
        "--output", "table"
    ])
else:
    print("❌ NAP 활성화 실패")
    print(result.stderr)

print()
print("💡 NAP 동작 방식:")
print("   - Pod가 스케줄될 수 없는 상황에서 AKS가 자동으로 적절한 VM 크기의 노드 풀을 생성")
print("   - 워크로드가 줄어들면 불필요한 노드 풀이 자동으로 축소/삭제")
print("   - Spot VM, GPU VM 등 다양한 VM 옵션을 자동으로 활용")

### 6.4 NAP 테스트: 워크로드 증가

NAP가 정상적으로 동작하는지 테스트하기 위해 Pod 수를 대폭 늘려 노드 부족 상황을 유도합니다.

> **참고**: HPA(Horizontal Pod Autoscaler)가 설정되어 있으면 수동 스케일링을 제한할 수 있으므로, 이 셀에서는 HPA를 먼저 삭제합니다.

In [ ]:
import subprocess

print("🔥 워크로드 대폭 증가 - NAP 테스트 시작")
print()

# HPA가 있으면 삭제 (수동 스케일링을 위해)
print("1️⃣ HPA 확인 및 삭제:")
hpa_check = subprocess.run([
    "kubectl", "get", "hpa", "myapp"
], capture_output=True, text=True)

if hpa_check.returncode == 0:
    print("   HPA가 존재합니다. NAP 테스트를 위해 삭제합니다...")
    subprocess.run(["kubectl", "delete", "hpa", "myapp"])
    print("   ✅ HPA 삭제 완료")
else:
    print("   ✅ HPA가 없습니다. 계속 진행합니다.")

print()

# 현재 노드 상태 확인
print("2️⃣ 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes"])

print()
print("3️⃣ 현재 노드 풀 목록:")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--query", "[].{Name:name, Count:count, Mode:mode}",
    "--output", "table"
])

print()
print("4️⃣ Pod 수를 100개로 증가 (NAP 트리거)...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=100"
])

print()
print("⏱️  잠시 대기 중 (Pod 생성 대기)...")
subprocess.run(["sleep", "5"])

print()
print("5️⃣ Pod 상태 확인:")
subprocess.run(["kubectl", "get", "pods", "-o", "wide"])

print()
print("6️⃣ Pending 상태 Pod 확인:")
pending_result = subprocess.run([
    "kubectl", "get", "pods",
    "--field-selector=status.phase=Pending"
], capture_output=True, text=True)

if "No resources found" not in pending_result.stdout:
    print(pending_result.stdout)
    print("⚠️  Pending 상태의 Pod가 있습니다!")
    print("   NAP가 자동으로 새 노드를 추가할 것입니다.")
else:
    print("   ✅ 모든 Pod가 Running 상태입니다.")
    print("   현재 노드 리소스가 충분합니다.")

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. 몇 분 후 섹션 6.5를 실행하여 NAP가 새 노드 풀을 생성했는지 확인")
print("   2. Pending 상태의 Pod가 있으면 NAP가 자동으로 노드를 추가합니다")
print("   3. NAP가 트리거되지 않았다면 현재 노드 용량이 충분한 것입니다")
print("=" * 80)


### 6.5 NAP 동작 확인

NAP에 의해 자동으로 생성된 노드 풀과 노드를 확인합니다.

In [ ]:
import subprocess

print("🔍 NAP 동작 확인 - 자동 생성된 노드 풀 및 노드 조회")
print()

print("📊 현재 노드 풀 목록 (NAP에 의해 생성된 풀 포함):")
subprocess.run([
    "az", "aks", "nodepool", "list",
    "--resource-group", RESOURCE_GROUP,
    "--cluster-name", AKS_CLUSTER_NAME,
    "--output", "table"
])

print()
print("📊 현재 노드 상태:")
subprocess.run(["kubectl", "get", "nodes", "-o", "wide"])

print()
print("📊 Pod 분산 상태:")
subprocess.run([
    "kubectl", "get", "pods",
    "-o", "custom-columns=NAME:.metadata.name,STATUS:.status.phase,NODE:.spec.nodeName"
])

print()
print("💡 확인 사항:")
print("   ✅ 새로운 노드 풀이 자동 생성되었는지 확인")
print("   ✅ Pending 상태였던 Pod들이 새 노드에 스케줄링되었는지 확인")
print("   ✅ NAP가 워크로드에 맞는 적절한 VM 크기를 선택했는지 확인")

### 6.6 NAP 테스트: 워크로드 감소

워크로드를 줄여서 NAP가 자동으로 노드 풀을 축소/삭제하는지 확인합니다.

In [ ]:
import subprocess

print("📉 워크로드 감소 - NAP 자동 축소 테스트")
print()

print("🔄 Pod 수를 다시 2개로 축소...")
subprocess.run([
    "kubectl", "scale", "deployment", "myapp",
    "--replicas=2"
])

print()
print("📊 현재 Pod 상태:")
subprocess.run(["kubectl", "get", "pods"])

print()
print("💡 NAP 자동 축소 확인 방법:")
print("   1. 몇 분 후 다시 노드 풀 목록을 확인")
print("   2. 불필요한 노드 풀이 자동으로 삭제되는지 모니터링")
print()
print("   실시간 모니터링 명령:")
print("   kubectl get nodes --watch")
print("   az aks nodepool list --resource-group", RESOURCE_GROUP, "--cluster-name", AKS_CLUSTER_NAME)

### 6.7 NAP 장점 정리

Node Auto Provisioning을 사용하면 다음과 같은 이점을 얻을 수 있습니다:

- ✅ **비용 최적화**: 워크로드에 따라 자동으로 스케일링되어 불필요한 리소스 비용 절감
- ✅ **리소스 효율성**: 다양한 VM 크기를 자동으로 선택하여 최적의 리소스 활용
- ✅ **운영 자동화**: 수동 노드 풀 관리 없이 자동으로 확장/축소
- ✅ **유연성**: GPU, Spot VM 등 다양한 VM 옵션 자동 활용
- ✅ **고가용성**: Pod가 Pending 상태에 머무르지 않고 즉시 스케줄링

> **참고**: 자세한 내용은 [Azure 공식 문서](https://learn.microsoft.com/en-us/azure/aks/node-autoprovision?tabs=azure-cli)를 참조하세요.

## 7. 정리

AKS 실습을 통해 클러스터 생성, 모니터링, 오토스케일링, Node Auto Provisioning 등 Kubernetes 기반 애플리케이션의 배포와 운영 자동화 전 과정을 GitHub Codespaces 환경에서 단계별로 실습했습니다.

### 실습한 내용 요약
- ✅ AKS 클러스터 생성 및 구성
- ✅ ACR 연동 및 이미지 배포
- ✅ LoadBalancer Service를 통한 외부 노출
- ✅ 수동 및 자동 스케일링 (HPA)
- ✅ Azure Monitor, Prometheus, Grafana를 통한 모니터링
- ✅ Node Auto Provisioning 테스트

> **다음 단계**: `03-cicd-automation.ipynb`에서 GitHub Actions와 Azure Pipelines를 사용한 CI/CD 자동화를 실습합니다.